In [6]:
import sys
import os
import importlib
import time
import random

from gensim import corpora, models, similarities
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel

from scipy.sparse import csr_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer #计算tfidf
from sklearn.feature_extraction.text import CountVectorizer  #计算df
from sklearn.feature_extraction.text import TfidfVectorizer  #“一步到位”
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.ensemble import RandomForestClassifier  
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib # to save
from sklearn import metrics

id_num = []
word_article = []
words_article = []
label = []
X_train = []
y_train = []
X_test = []
y_test = []

load_start = time.time()
importlib.reload(sys)
with open('train_set.csv','r',encoding = 'utf-8') as f:
    lines = f.readlines()[1:]
#     random.shuffle(lines) #注意用法
    for line in lines[1:10000]:
        line = line.split(',')
        X_train.append(line[2])
        y_train.append(line[3])
    for line in lines[100000:]: #划分出测试集
        line = line.split(',')
        X_test.append(line[2])
        y_test.append(line[3])
words = [[] for i in range(len(X_train) + len(X_test))]   
i = 0
for i in range(len(X_train)):
    for j in X_train[i].split(' '):
        words[i].append(j)
#     corpus[i].append(j for j in X_train[i].split(' ')) #错的？
i += 1
ii =0
for i in range(i,i+len(X_test)):
    for j in X_test[ii].split(' '):
        words[i].append(j)
    ii += 1
# print(corpus[-1])
print("Time consumption on loading:",time.time() - load_start)

train_start = time.time()
dictionary = corpora.Dictionary(words)
# print(dictionary)
corp = [ dictionary.doc2bow(word) for word in words ] #词频统计+one hot
tfidf_model = models.TfidfModel(corpus=words, dictionary=dictionary)#这里用words和corp训练结果是一样的
corp_tfidf = [tfidf_model[doc] for doc in corp]
# print(len(corp_tfidf), corp_tfidf[0])
lda = LdaModel(corpus=corp_tfidf, id2word=dictionary, num_topics=2000) #可以直接换成LsiModel
corp_lda = [lda[doc] for doc in corp]
data = []
rows = []
cols = []
line_count = 0
for line in corp_lda:  # corp_lda 是之前由gensim生成的lsi向量
    for elem in line:
        rows.append(line_count)
        cols.append(elem[0])
        data.append(elem[1])
    line_count += 1
lda_sparse_matrix = csr_matrix((data,(rows,cols))) 
lda_matrix = lda_sparse_matrix.toarray()  
print(lda_matrix.shape)
X_train = lda_matrix[0:9999]



# vect = TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.95, use_idf=1, smooth_idf=1, sublinear_tf=1)
# vect.fit(X_train) # 或者直接X_train = vect.fit_transform(X_train)
# X_train = vect.transform(X_train)




# X_train = X_train.toarray() #由稀疏矩阵变成ndarray类型的完整矩阵。。？
# pca = PCA(n_components = 10000)
# print(X_train.shape)
# X_train = pca.fit_transform(X_train)
# print(X_train.shape)
print("Time consumption on training1:",time.time() - train_start)


clf = svm.LinearSVC(C=0.5)
# clf = svm.SVC(kernel='linear', C=1, )
# clf = SGDClassifier(loss="modified_huber", )#penalty=”elasticnet”: L2和L1的convex组合; (1 - l1_ratio) * L2 + l1_ratio * L1
clf.fit(X_train, y_train)
scores = cross_val_score(clf, X_train, y_train , cv=5)
# print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
# joblib.dump(clf, 'lineSVM_20000__.m')
print("Time consumption on training1+2:",time.time() - train_start)


test_start = time.time()
# clf = joblib.load('lineSVM_20000__.m')
prediction = clf.predict(X_train)
print(metrics.classification_report(y_train,prediction))
X_test = vect.transform(X_test)
# X_test = X_test.toarray()
print(X_test.shape)
prediction = clf.predict(X_test) 
print(metrics.classification_report(y_test,prediction))
# print(metrics.confusion_matrix(y_test,prediction))
print("Time consumption on testing:",time.time() - test_start)

Time consumption on loading: 3.482760190963745


/home/amax/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)
/home/amax/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:676: RuntimeWarning: overflow encountered in true_divide
  gammad = self.alpha + expElogthetad * np.dot(cts / phinorm, expElogbetad.T)


(12276, 1999)
Time consumption on training1: 1295.7346184253693
Accuracy: 0.50 (+/- 0.02)
Time consumption on training1+2: 1298.724730014801
             precision    recall  f1-score   support

         1
       0.64      0.17      0.27       537
        10
       0.46      0.57      0.51       478
        11
       0.48      0.27      0.35       337
        12
       0.63      0.24      0.35       531
        13
       0.46      0.61      0.53       777
        14
       0.57      0.71      0.63       647
        15
       0.45      0.61      0.52       752
        16
       0.47      0.05      0.08       306
        17
       0.65      0.47      0.55       287
        18
       0.66      0.79      0.72       680
        19
       0.43      0.10      0.16       577
         2
       0.56      0.73      0.63       279
         3
       0.47      0.59      0.52       808
         4
       0.68      0.06      0.11       369
         5
       0.51      0.18      0.27       223
         6

NotFittedError: TfidfVectorizer - Vocabulary wasn't fitted.